## Entrenamiento de Arbol de decision
-   Crear árbol de decisión de clasificación:
    -   Sacamos acurracy (si el dataset es muy grande sacar además recall, precisión y F1-Score)
-   Realizar ajuste automático de hiperparámetros
-   En función de la salida intentamos ajustar al detalle
-   Mostramos e interpretamos los resultados

In [32]:
# Importación de librerias 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn import tree 
from sklearn.preprocessing import StandardScaler

### Limpieza y carga de datos datos

In [33]:
# Cargamos el CSV
df = pd.read_csv("banking.csv")

# Analizamos el contenido del dataset
# print(df.info())
# print(df.describe())

# Obtenemos la cantidad de valores únicos por columna para averiguar las variables categóricas 
# for col in df.columns:
#     print(f"{col}: {df[col].unique()}")
#     print(f"{col}: {len(df[col].unique())}")

# Definimos las variables categóricas
categoricos = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]

# Resumir la columna educación
df["education"] = np.where((df["education"] == 'basic.4y') | (df["education"] == 'basic.6y') | (df["education"] == 'basic.9y'), 
                           'Basic', 
                           df["education"])


### Preprocesamiento de los datos y entrenamiento del modelo

Primer entrenamiento del modelo con hiperparámetros por defecto

In [ ]:
# Encodeamos las variables categóricas mediante OneHotEncoder
df = pd.get_dummies(df, columns=categoricos, drop_first=True)

# Definimos las variables predictoras y objetivo
X = df.drop(columns=["y"])
y = df["y"]

# Normalizamos las variables numéricas mediante StandarScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definimos el modelo
model = DecisionTreeClassifier(random_state=42)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy del modelo sin ajustar hiperparámetros: 88.78%


Segundo entrenamiento del modelo mediante la búsqueda automática de hiperparámetros

In [35]:
# Búsqueda automática de hiperparámetros
# Definimos los hiperparámetros sobre los que queremos que realice pruebas el GridSearch
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'splitter' : ['best', 'random']
}
# Definimos el objeto de GridSearchCV mediante el modelo, los hiperparámetros, el nº de iteraciones y la métrica de evuliación de los modelos
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Entrenamos el modelo
grid_search.fit(X_train, y_train)

# Imprimimos los mejores parámetros
print(f"Mejores parámetros: {grid_search.best_params_}")

# Evaluar modelo
accuracy_grid = grid_search.score(X_test, y_test)
print(f"Accuracy del modelo sin ajustar hiperparámetros: {accuracy_grid * 100:.2f}%")

Mejores parámetros: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
Accuracy del modelo sin ajustar hiperparámetros: 91.16%


Mediante la búsqueda automática de hiperparámetros determinamos que los mejores son:
-   ``'criterion': 'gini'``
-   ``'max_depth': 5 ``
-   ``'min_samples_leaf': 1 ``
-   ``'min_samples_split': 2``
-   ``'splitter': 'best'``